# Loop 7 Analysis: Rotation Optimization and bbox3

The evaluator identified two key techniques NOT yet tried:
1. **Rotation optimization (fix_direction)** - Optimize the rotation angle of the entire configuration
2. **bbox3 optimizer** - A fundamentally different optimization approach

Let's analyze these approaches.

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
from shapely.geometry import Polygon
from shapely import affinity

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    vertices = [(tx * cos_a - ty * sin_a + x, tx * sin_a + ty * cos_a + y) for tx, ty in zip(TX, TY)]
    return Polygon(vertices)

print('Functions defined')

In [ ]:
# Load the current best baseline
df = pd.read_csv('/home/code/external_data/saspav/santa-2025.csv')
print(f'Loaded {len(df)} rows')
print(df.head())

In [ ]:
# Implement rotation optimization (fix_direction)
def calculate_bbox_side_at_angle(angle_deg, points):
    """Calculate bounding box side length at a given rotation angle."""
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, s], [-s, c]])
    rotated_points = points @ rot_matrix.T
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation_for_n(df, n):
    """Optimize rotation angle for configuration N."""
    prefix = f'{n:03d}_'
    trees = df[df['id'].str.startswith(prefix)]
    
    if len(trees) != n:
        return None, None, None
    
    # Get all polygon vertices
    all_points = []
    for _, row in trees.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        poly = create_tree_polygon(x, y, deg)
        all_points.extend(list(poly.exterior.coords))
    
    points = np.array(all_points)
    
    # Get convex hull for efficiency
    try:
        hull_points = points[ConvexHull(points).vertices]
    except:
        hull_points = points
    
    # Calculate initial side
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    
    # Optimize rotation
    result = minimize_scalar(
        lambda a: calculate_bbox_side_at_angle(a, hull_points),
        bounds=(0, 90),
        method='bounded'
    )
    
    best_angle = result.x
    best_side = result.fun
    improvement = initial_side - best_side
    
    return initial_side, best_side, best_angle, improvement

print('Rotation optimization function defined')

In [ ]:
# Test rotation optimization on a few N values
test_ns = [1, 10, 50, 100, 150, 200]
print('Testing rotation optimization on selected N values:\n')

for n in test_ns:
    result = optimize_rotation_for_n(df, n)
    if result[0] is not None:
        initial, best, angle, improvement = result
        print(f'N={n:3d}: Initial={initial:.6f}, Best={best:.6f}, Angle={angle:.4f}°, Improvement={improvement:.9f}')

In [ ]:
# Apply rotation optimization to ALL N values and calculate total improvement
print('Applying rotation optimization to all N values...')

total_improvement = 0
improvements = []

for n in range(1, 201):
    result = optimize_rotation_for_n(df, n)
    if result[0] is not None:
        initial, best, angle, improvement = result
        # Score contribution: side^2 / n
        initial_score = initial**2 / n
        best_score = best**2 / n
        score_improvement = initial_score - best_score
        total_improvement += score_improvement
        improvements.append({
            'n': n,
            'initial_side': initial,
            'best_side': best,
            'angle': angle,
            'side_improvement': improvement,
            'score_improvement': score_improvement
        })

print(f'\nTotal score improvement from rotation: {total_improvement:.9f}')
print(f'Current score: 70.659959')
print(f'New score after rotation: {70.659959 - total_improvement:.6f}')

In [ ]:
# Show top 10 N values with most improvement from rotation
import pandas as pd
imp_df = pd.DataFrame(improvements)
imp_df = imp_df.sort_values('score_improvement', ascending=False)
print('Top 10 N values with most improvement from rotation:')
print(imp_df.head(10).to_string(index=False))

In [ ]:
# Summary of rotation optimization potential
print('='*60)
print('ROTATION OPTIMIZATION SUMMARY')
print('='*60)
print(f'Total score improvement: {total_improvement:.9f}')
print(f'N values with improvement > 1e-6: {len(imp_df[imp_df.score_improvement > 1e-6])}')
print(f'N values with improvement > 1e-9: {len(imp_df[imp_df.score_improvement > 1e-9])}')
print(f'Average improvement per N: {total_improvement/200:.12f}')
print('='*60)
print(f'\nConclusion: Rotation optimization provides {total_improvement:.9f} improvement.')
print(f'This is {"significant" if total_improvement > 0.001 else "negligible"} compared to the 1.74 gap to target.')